In [7]:
from pathlib import Path
import json

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

import matplotlib.pyplot as plt
import seaborn as sns

import biopsykit.carwatch_logs.log_actions as la
from biopsykit import carwatch_logs as cl
from biopsykit.io import carwatch_logs

import biopsykit as bp

import datetime

%load_ext autoreload
%autoreload 2
%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load Self-Report Data

In [4]:
export_path = Path("../00_general").joinpath("exports")

In [8]:
data_raw = pd.read_csv(export_path.joinpath("cortisol_samples_cleaned.csv"))
data_raw = data_raw[["subject", "condition", "night", "sample", "time", "cortisol"]]

data_raw.head()

,subject,condition,night,sample,time,cortisol
0,AB19E,Known Alarm,1,S0,0.0,4.27
1,AB19E,Known Alarm,1,S1,15.0,14.86
2,AB19E,Known Alarm,1,S2,30.0,25.97
3,AB19E,Known Alarm,1,S3,45.0,24.26
4,AB19E,Known Alarm,1,S4,60.0,17.13


## Load App Logs

In [9]:
deploy_type = "develop"

In [11]:
# build path to data folder
config_dict = json.load(Path("../config.json").open(encoding="utf-8"))
base_path = Path("..").joinpath(config_dict[deploy_type]["base_path"])
base_path

PosixPath('/Users/Richer/Documents/PhD/Projects/HealthPsychology/CARWatch/Data')

In [14]:
logs_path = base_path.joinpath("app_logs/cleaned_manual")
logs = carwatch_logs.load_logs_all_subjects(logs_path, has_subject_folders=False, return_df=False)

  0%|          | 0/65 [00:00<?, ?it/s]

## Load Cleaned App Wakeup Onsets

In [107]:
app_wakeup_path = Path("../../../carwatch-data/App_Logs/app_data_wakeup.xlsx")

app_data = pd.read_excel(app_wakeup_path)
app_data.index = app_data['Code']

#app_data.drop(['Code'], axis=1, inplace=True)

app_data.head()

,Code,0,1
Code,,,
CE10B,CE10B,04:48:53,05:40:54
EA09H,EA09H,09:17:47,08:11:54
EM16N,EM16N,08:51:12,NaN
KA17A,KA17A,06:39:00,05:41:00
KA19E,KA19E,NaN,NaN


In [108]:
app_data_long = app_data.melt(['Code'], [0,1], 'night', 'time')
app_data_long.set_index(['Code', 'night'], inplace=True)
app_data_long.dropna(inplace=True)


app_data_long.head()

,,time
Code,night,
CE10B,0,04:48:53
EA09H,0,09:17:47
EM16N,0,08:51:12
KA17A,0,06:39:00
LB21E,0,08:57:41


## Load Self-Report Wakeup Onsets

In [109]:
self_report_wake_onset_path = Path("../../../carwatch-data/Questionnaire_Data/Questionnaire_Data_CARWatch.csv")

In [110]:
sr_wake_onset = pd.read_csv(self_report_wake_onset_path)
sr_wake_onset.head()

,subject,id,birthYear,age,gender,education,oralContraceptive,smoking,weight,height,...,cortTime_N2_S4,cortTime_N3_SA,sleepOnsetSensor_N1,wakeOnsetSensor_N1,sleepOnsetSensor_N2,wakeOnsetSensor_N2,condition_N1,condition_N2,SubjectiveSleepQuality_N1,SubjectiveSleepQuality_N2
0,AB19E,19,1980,40,2,3,NaN,NaN,NaN,NaN,...,10:00:00,22:45:00,23:06:22,05:56:22,22:22:44,08:56:44,1,1,3.0,3.0
1,AB31R,114,1970,50,2,4,NaN,NaN,NaN,NaN,...,06:45:00,22:15:00,NaN,NaN,NaN,NaN,3,1,1.0,1.0
2,AC12E,18,1997,23,2,4,NaN,NaN,NaN,NaN,...,08:34:00,22:55:00,23:52:16,06:57:16,01:19:36,07:27:36,1,1,2.0,1.0
3,AH10Y,39,2000,20,2,4,NaN,NaN,NaN,NaN,...,06:33:00,22:05:00,01:14:43,07:26:43,23:29:51,05:27:51,1,1,2.0,3.0
4,AK12I,50,1995,25,1,5,2.0,1.0,87.0,160.0,...,09:51:00,22:18:00,01:20:31,09:28:31,00:23:59,09:07:59,3,3,1.0,3.0


In [111]:
# Set index
sr_wake_onset.rename({'subject': 'Code'}, axis=1, inplace = True)

sr_wake_onset = sr_wake_onset[['Code', 'wakeOnsetSelfreport_N1', 'wakeOnsetSelfreport_N2']]
#sr_wake_onset.set_index('Code', inplace= True)

sr_wake_onset.head()

,Code,wakeOnsetSelfreport_N1,wakeOnsetSelfreport_N2
0,AB19E,05:45:00,NaN
1,AB31R,05:40:00,05:50:00
2,AC12E,07:00:00,07:30:00
3,AH10Y,07:30:00,05:30:00
4,AK12I,09:24:00,08:51:00


In [112]:
wake_onset_long = pd.melt(sr_wake_onset, id_vars=['Code'], value_vars=['wakeOnsetSelfreport_N1', 'wakeOnsetSelfreport_N2'], var_name='night', value_name='time')
wake_onset_long.replace({'wakeOnsetSelfreport_N1' : 0, 'wakeOnsetSelfreport_N2': 1}, inplace=True)
wake_onset_long.set_index(['Code', 'night'], inplace=True)

wake_onset_long.head()

,,time
Code,night,
AB19E,0,05:45:00
AB31R,0,05:40:00
AC12E,0,07:00:00
AH10Y,0,07:30:00
AK12I,0,09:24:00


## Extract Saliva Sample Times From App Logs

In [115]:
def get_timestamp(df, saliva_id):
    return datetime.datetime.strptime(df.loc[df['saliva_id']==saliva_id].iloc[0].name.strftime("%H:%M:%S"), "%H:%M:%S").time()

def get_datetime(timestring):
    return datetime.datetime.strptime(timestring, "%H:%M:%S").time()

def get_from_json(row):
    json_extra = json.loads(row.extras)
    return json_extra.get('saliva_id')

for subject, subject_log in logs.items():
    log_subject = carwatch_logs.LogData(subject_log)
    
    finished_day = log_subject.finished_days

    df = cl.log_data.get_logs_for_action(log_subject, la.barcode_scanned)

    for day in range(2):
        try:
            df_day = df[df.index.date == finished_day[day].date()]
            df_day['saliva_id'] = df_day.apply(lambda row: get_from_json(row), axis=1)
                
            if pd.isna(app_data.loc[subject, day]):
                continue
            
            T0 = get_datetime(app_data.loc[subject, day])
            T0_delta = datetime.timedelta(hours=T0.hour, minutes=T0.minute, seconds=T0.second)
            
            for saliva_id in range(5):
                T = get_timestamp(df_day, saliva_id)
                T_delta = datetime.timedelta(hours=T.hour, minutes=T.minute, seconds=T.second)

                data_raw.loc[(data_raw['subject']==subject) & (data_raw['night']==day) & (data_raw['sample']=='S'+str(saliva_id)), 'time_app'] = (T_delta-T0_delta).total_seconds() / 60
                #data_raw.loc[(data_raw['subject']==subject) & (data_raw['night']==day) & (data_raw['sample']=='S'+str(saliva_id)), 'wake_onset_app'] = app_data_long.xs((subject, day))                

        except IndexError:
            pass
            
            

data = data_raw.rename(columns={'time':'self_report','time_app':'app'})

In [116]:
data['naive'] = np.empty(data.shape[0])

for sample in range(5):
    data["naive"].loc[data["sample"]=="S"+str(sample)] = sample*15
    
data.head()

,condition,subject,chronotype,MEQ,night,within_ideal_bed_time,wakeup_source,date,weekend,wakeup_hour,sample,cortisol,self_report,app,naive
0,Spontaneous,AB31R,Intermediate,56.0,0,False,Spontaneous,2020-01-08,Weekday,5,S0,10.28,0.0,NaN,0.0
1,Spontaneous,AB31R,Intermediate,56.0,0,False,Spontaneous,2020-01-08,Weekday,5,S1,12.37,17.0,NaN,15.0
2,Spontaneous,AB31R,Intermediate,56.0,0,False,Spontaneous,2020-01-08,Weekday,5,S2,10.41,32.0,NaN,30.0
3,Spontaneous,AB31R,Intermediate,56.0,0,False,Spontaneous,2020-01-08,Weekday,5,S3,10.22,47.0,NaN,45.0
4,Spontaneous,AB31R,Intermediate,56.0,0,False,Spontaneous,2020-01-08,Weekday,5,S4,8.21,62.0,NaN,60.0


In [117]:
# Optional: drop all subjects with self report only

data_clean = data.dropna()
self_report_only = data[~data.index.isin(data_clean.index)]

self_report_only.head()

,condition,subject,chronotype,MEQ,night,within_ideal_bed_time,wakeup_source,date,weekend,wakeup_hour,sample,cortisol,self_report,app,naive
0,Spontaneous,AB31R,Intermediate,56.0,0,False,Spontaneous,2020-01-08,Weekday,5,S0,10.28,0.0,NaN,0.0
1,Spontaneous,AB31R,Intermediate,56.0,0,False,Spontaneous,2020-01-08,Weekday,5,S1,12.37,17.0,NaN,15.0
2,Spontaneous,AB31R,Intermediate,56.0,0,False,Spontaneous,2020-01-08,Weekday,5,S2,10.41,32.0,NaN,30.0
3,Spontaneous,AB31R,Intermediate,56.0,0,False,Spontaneous,2020-01-08,Weekday,5,S3,10.22,47.0,NaN,45.0
4,Spontaneous,AB31R,Intermediate,56.0,0,False,Spontaneous,2020-01-08,Weekday,5,S4,8.21,62.0,NaN,60.0


In [118]:
id_vars = ['subject', 'condition', 'chronotype', 'MEQ', 'night', 'wakeup_source', 'weekend', 'date', 'wakeup_hour', 'sample', 'cortisol']
value_vars = ['self_report','app','naive']
cort_long = data_clean.melt(id_vars=id_vars, value_vars=value_vars, var_name='log_mode', value_name='time')

cort_long_self_report_only = self_report_only.melt(id_vars =id_vars, value_vars=value_vars, var_name='log_mode', value_name='time')

cort_long_self_report_only = cort_long_self_report_only.replace({'self_report': 'self_report_only', 'naive': 'naive_sr_only'})
                                    
cort_long = pd.concat([cort_long, cort_long_self_report_only])
cort_long.dropna(inplace=True)
       
cort_long.head()

,subject,condition,chronotype,MEQ,night,wakeup_source,weekend,date,wakeup_hour,sample,cortisol,log_mode,time
0,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,S0,5.88,self_report,0.0
1,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,S1,12.54,self_report,15.0
2,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,S2,12.91,self_report,30.0
3,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,S3,12.30,self_report,45.0
4,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,S4,10.38,self_report,60.0


In [119]:
cort_long['time_diff'] = np.empty(cort_long.shape[0])

for sample in range(5):
    cort_long["time_diff"].loc[cort_long["sample"]=="S"+str(sample)] = cort_long.loc[cort_long["sample"]=="S"+str(sample)].apply(lambda row: row["time"]-15*sample, axis=1)
    
cort_long.head()

,subject,condition,chronotype,MEQ,night,wakeup_source,weekend,date,wakeup_hour,sample,cortisol,log_mode,time,time_diff
0,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,S0,5.88,self_report,0.0,0.0
1,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,S1,12.54,self_report,15.0,0.0
2,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,S2,12.91,self_report,30.0,0.0
3,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,S3,12.30,self_report,45.0,0.0
4,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,S4,10.38,self_report,60.0,0.0


In [120]:
# reindex to match all subjects

app_data_long = app_data_long.reindex_like(wake_onset_long)

In [121]:
cort_long["wakeup_self_report"] = cort_long.apply(lambda row: wake_onset_long.xs((row['subject'], row['night'])), axis=1)

cort_long["wakeup_app"] = cort_long.apply(lambda row: app_data_long.xs((row['subject'], row['night'])), axis=1)

cort_long.head()

,subject,condition,chronotype,MEQ,night,wakeup_source,weekend,date,wakeup_hour,sample,cortisol,log_mode,time,time_diff,wakeup_self_report,wakeup_app
0,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,S0,5.88,self_report,0.0,0.0,07:31:00,07:31:16
1,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,S1,12.54,self_report,15.0,0.0,07:31:00,07:31:16
2,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,S2,12.91,self_report,30.0,0.0,07:31:00,07:31:16
3,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,S3,12.30,self_report,45.0,0.0,07:31:00,07:31:16
4,BC05R,Spontaneous,Intermediate,47.0,0,Spontaneous,Weekend,2019-12-07,7,S4,10.38,self_report,60.0,0.0,07:31:00,07:31:16


In [122]:
count = pd.DataFrame(cort_long.groupby('log_mode').size()/5)
count.rename(columns={0:'number of samples'}, inplace=True)
count

,number of samples
log_mode,
app,50.0
naive,50.0
naive_sr_only,92.0
self_report,50.0
self_report_only,92.0


In [123]:
idx = ['subject', 'condition', 'chronotype', 'MEQ', 'night', 'wakeup_source', 'weekend', 'date', 'wakeup_hour', 'log_mode', 'wakeup_self_report', 'wakeup_app', 'sample']
cort_long.set_index(idx, inplace = True)

cort_long.head()

cortisol  \
subject condition   chronotype   MEQ  night wakeup_source weekend date       wakeup_hour log_mode    wakeup_self_report wakeup_app sample             
BC05R   Spontaneous Intermediate 47.0 0     Spontaneous   Weekend 2019-12-07 7           self_report 07:31:00           07:31:16   S0          5.88   
                                                                                                                                   S1         12.54   
                                                                                                                                   S2         12.91   
                                                                                                                                   S3         12.30   
                                                                                                                                   S4         10.38   

                                                                                                                                           time  \
subject condition   chronotype   MEQ  night wakeup_source weekend date       wakeup_hour log_mode    wakeup_self_report wakeup_app sample         
BC05R   Spontaneous Intermediate 47.0 0     Spontaneous   Weekend 2019-12-07 7           self_report 07:31:00           07:31:16   S0       0.0   
                                                                                                                                   S1      15.0   
                                                                                                                                   S2      30.0   
                                                                                                                                   S3      45.0   
                                                                                                                                   S4      60.0   

                                                                                                                                           time_diff  
subject condition   chronotype   MEQ  night wakeup_source weekend date       wakeup_hour log_mode    wakeup_self_report wakeup_app sample             
BC05R   Spontaneous Intermediate 47.0 0     Spontaneous   Weekend 2019-12-07 7           self_report 07:31:00           07:31:16   S0            0.0  
                                                                                                                                   S1            0.0  
                                                                                                                                   S2            0.0  
                                                                                                                                   S3            0.0  
                                                                                                                                   S4            0.0

In [124]:
# export
cort_long.to_csv(export_path.joinpath('cortisol_samples_app_cleaned.csv'), index=True)

## Compute Cortisol Features For App Times

In [25]:
cort_auc = bp.saliva.auc(cort_long, remove_s0=False)
cort_auc.head()

saliva_feature                                                                                             cortisol_auc_g  \
subject condition   chronotype   MEQ  night wakeup_source weekend date       wakeup_hour log_mode                           
AB31R   Spontaneous Intermediate 56.0 0     Spontaneous   Weekday 2020-01-08 5           naive_sr_only            633.675   
                                                                                         self_report_only         656.325   
AC12E   Known Alarm Intermediate 46.0 0     Alarm         Weekday 2019-11-18 7           naive_sr_only            700.425   
                                                                                         self_report_only         751.950   
                                      1     Alarm         Weekday 2019-11-19 7           naive_sr_only            925.950   

saliva_feature                                                                                             cortisol_auc_i  
subject condition   chronotype   MEQ  night wakeup_source weekend date       wakeup_hour log_mode                          
AB31R   Spontaneous Intermediate 56.0 0     Spontaneous   Weekday 2020-01-08 5           naive_sr_only             16.875  
                                                                                         self_report_only          18.965  
AC12E   Known Alarm Intermediate 46.0 0     Alarm         Weekday 2019-11-18 7           naive_sr_only            377.625  
                                                                                         self_report_only         402.250  
                                      1     Alarm         Weekday 2019-11-19 7           naive_sr_only            436.950

In [26]:
cort_inc = bp.saliva.max_increase(cort_long, remove_s0=False)
cort_inc.head()

saliva_feature                                                                                             cortisol_max_inc
subject condition   chronotype   MEQ  night wakeup_source weekend date       wakeup_hour log_mode                          
AB31R   Spontaneous Intermediate 56.0 0     Spontaneous   Weekday 2020-01-08 5           naive_sr_only                 2.09
                                                                                         self_report_only              2.09
AC12E   Known Alarm Intermediate 46.0 0     Alarm         Weekday 2019-11-18 7           naive_sr_only                 9.05
                                                                                         self_report_only              9.05
                                      1     Alarm         Weekday 2019-11-19 7           naive_sr_only                11.67

In [27]:
cort_slope = bp.saliva.slope(cort_long, sample_idx=[0, 3])
cort_slope = cort_slope.join(bp.saliva.slope(cort_long, sample_idx=[0, 4]))
#cort_slope = cort_slope.join(bp.saliva.slope(cort_long, sample_idx=[2, 4], biomarker_type='cortisol'))
cort_slope.head()

saliva_feature                                                                                             cortisol_slopeS0S3  \
subject condition   chronotype   MEQ  night wakeup_source weekend date       wakeup_hour log_mode                               
AB31R   Spontaneous Intermediate 56.0 0     Spontaneous   Weekday 2020-01-08 5           naive_sr_only              -0.001333   
                                                                                         self_report_only           -0.001277   
AC12E   Known Alarm Intermediate 46.0 0     Alarm         Weekday 2019-11-18 7           naive_sr_only               0.188667   
                                                                                         self_report_only            0.188667   
                                      1     Alarm         Weekday 2019-11-19 7           naive_sr_only               0.259333   

saliva_feature                                                                                             cortisol_slopeS0S4  
subject condition   chronotype   MEQ  night wakeup_source weekend date       wakeup_hour log_mode                              
AB31R   Spontaneous Intermediate 56.0 0     Spontaneous   Weekday 2020-01-08 5           naive_sr_only              -0.034500  
                                                                                         self_report_only           -0.033387  
AC12E   Known Alarm Intermediate 46.0 0     Alarm         Weekday 2019-11-18 7           naive_sr_only               0.113167  
                                                                                         self_report_only            0.104462  
                                      1     Alarm         Weekday 2019-11-19 7           naive_sr_only               0.129000

In [28]:
cort_max = pd.DataFrame(cort_long['cortisol'].unstack('sample').max(axis=1), columns=['cortisol_cmax'])
cort_max.head()

cortisol_cmax
subject condition   chronotype   MEQ  night wakeup_source weekend date       wakeup_hour log_mode                       
AB31R   Spontaneous Intermediate 56.0 0     Spontaneous   Weekday 2020-01-08 5           naive_sr_only             12.37
                                                                                         self_report_only          12.37
AC12E   Known Alarm Intermediate 46.0 0     Alarm         Weekday 2019-11-18 7           naive_sr_only             14.43
                                                                                         self_report_only          14.43
                                      1     Alarm         Weekday 2019-11-19 7           naive_sr_only             19.82

In [29]:
cort_cini = cort_long.xs('S0', level='sample')[['cortisol']]
cort_cini.columns = ["cortisol_cini"]
cort_cini.head()

cortisol_cini
subject condition     chronotype   MEQ  night wakeup_source weekend date       wakeup_hour log_mode                  
BC05R   Spontaneous   Intermediate 47.0 0     Spontaneous   Weekend 2019-12-07 7           self_report        5.88000
                                        1     Spontaneous   Weekend 2019-12-08 8           self_report        5.19000
BU07E   Known Alarm   Evening      61.0 0     Alarm         Weekday 2019-12-06 6           self_report        4.17000
CC09K   Unknown Alarm Intermediate 58.0 0     Alarm         Weekday 2019-12-02 6           self_report        4.00000
CE10B   Known Alarm   Evening      64.0 0     Alarm         Weekday 2019-11-12 4           self_report        7.70715

In [30]:
cort_feat = pd.concat([cort_auc, cort_inc, cort_slope, cort_cini, cort_max], axis=1)
cort_feat = pd.DataFrame(cort_feat.stack(), columns=['cortisol'])
cort_feat.index = cort_feat.index.set_names(cort_feat.index.names[:-1] + ['saliva_feature'])
cort_feat.head()

cortisol
subject condition   chronotype   MEQ  night wakeup_source weekend date       wakeup_hour log_mode      saliva_feature                
AB31R   Spontaneous Intermediate 56.0 0     Spontaneous   Weekday 2020-01-08 5           naive_sr_only cortisol_auc_g      633.675000
                                                                                                       cortisol_auc_i       16.875000
                                                                                                       cortisol_max_inc      2.090000
                                                                                                       cortisol_slopeS0S3   -0.001333
                                                                                                       cortisol_slopeS0S4   -0.034500

In [31]:
# export
cort_feat.to_csv(export_path.joinpath('cortisol_features_app_cleaned.csv'))
